In [1]:
install.packages('DT')
install.packages("bigrquery")

also installing the dependencies ‘lazyeval’, ‘crosstalk’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependency ‘rapidjsonr’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library(tidyverse)
library(DBI)
library(stringr)
library(DT)
library(lubridate)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [18]:
con <- dbConnect(
  bigrquery::bigquery(),
  project = "cal-itp-data-infra" ,
  dataset = "mart_gtfs" ,
  page_size = 4000
#  billing = "billing"
  )

In [4]:
base_tts_contracts_idmap_query = 
"
WITH
latest AS (
    

WITH ranked AS (
    SELECT *,
        DENSE_RANK() OVER (
            
            PARTITION BY dt
            
            ORDER BY ts DESC) as rank,
        -- TODO: this is probably a bit fragile, but lets us fake historical Airtable data
        MIN(ts) OVER () AS universal_first_val
    FROM `cal-itp-data-infra`.`external_airtable`.`transit_technology_stacks__contracts`
    QUALIFY rank = 1
)
SELECT * FROM ranked


),

base_tts_organizations_ct_organizations_map AS (
    SELECT * FROM `cal-itp-data-infra`.`staging`.`base_tts_organizations_ct_organizations_map`
),

base_tts_services_ct_services_map AS (
    SELECT * FROM `cal-itp-data-infra`.`staging`.`base_tts_services_ct_services_map`
),

mapped_org_ids AS (
    SELECT
        id,
        ARRAY_AGG(map_holder.ct_key IGNORE NULLS) AS contract_holder,
        ARRAY_AGG(map_vendor.ct_key IGNORE NULLS) AS contract_vendor,
        ARRAY_AGG(map_services.ct_key IGNORE NULLS) AS covered_services,
    dt
    FROM latest
    LEFT JOIN UNNEST(latest.contract_holder) AS unnested_contract_holder
    LEFT JOIN UNNEST(latest.contract_vendor) AS unnested_contract_vendor
    LEFT JOIN UNNEST(latest.covered_services) AS unnested_covered_services
    LEFT JOIN base_tts_organizations_ct_organizations_map AS map_holder
        ON unnested_contract_holder = map_holder.tts_key
        AND dt = map_holder.tts_date
    LEFT JOIN base_tts_organizations_ct_organizations_map AS map_vendor
        ON unnested_contract_vendor = map_vendor.tts_key
        AND dt = map_vendor.tts_date
    LEFT JOIN base_tts_services_ct_services_map AS map_services
        ON unnested_covered_services = map_services.tts_key
        AND dt = map_services.tts_date
    GROUP BY id, dt
),

base_tts_contracts_idmap AS (
    SELECT
        r.* EXCEPT(contract_holder, contract_vendor, covered_services),
        map.contract_holder,
        map.contract_vendor,
        map.covered_services
    FROM latest as r
    LEFT JOIN mapped_org_ids AS map
        USING (id, dt)
)

SELECT * FROM base_tts_contracts_idmap
"

In [19]:
base_tts_contracts_idmap = dbGetQuery(con, base_tts_contracts_idmap_query)

In [13]:
stg_transit_database__contracts_query =
'
WITH

once_daily_contracts AS (
    SELECT *
    -- have to use base table to get the california transit base organization record ids
    FROM base_tts_contracts_idmap
),

stg_transit_database__contracts AS (
    SELECT
        id,
        

CASE
    WHEN TRIM(name) = ""
        THEN NULL
    ELSE TRIM(name)
END

 AS name,
        unnested_contract_holder AS contract_holder_organization_key,
        unnested_contract_vendor AS contract_vendor_organization_key,
        covered_components,
        covered_services,
        value,
        start_date,
        end_date,
        renewal_option,
        notes,
        contract_name AS contract_name_notes,
        attachments,
        dt
    FROM base_tts_contracts_idmap
    LEFT JOIN UNNEST(once_daily_contracts.contract_holder) AS unnested_contract_holder
    LEFT JOIN UNNEST(once_daily_contracts.contract_vendor) AS unnested_contract_vendor
)

SELECT * FROM stg_transit_database__contracts
'

In [14]:
stg_transit_database__contracts = dbGetQuery(con, stg_transit_database__contracts_query)

ERROR: [1m[33mError[39m in `BigQueryResult()`:[22m
[1m[22m[33m![39m Job cal-itp-data-infra.job_42CDn0eZQlxkgv-HIgmK3BmROS6U.us-west2
  failed
[31m✖[39m Not found: Table cal-itp-data-infra:mart_gtfs.base_tts_contracts_idmap was
  not found in location us-west2 [notFound]


In [ ]:
int_transit_database__contracts_dim_query = 
"
WITH latest_contracts AS (
    

WITH ranked AS (
    SELECT *,
        DENSE_RANK() OVER (
            
            ORDER BY dt DESC) as rank,
        -- TODO: this is probably a bit fragile, but lets us fake historical Airtable data
        MIN(dt) OVER () AS universal_first_val
    FROM `cal-itp-data-infra`.`staging`.`stg_transit_database__contracts`
    QUALIFY rank = 1
)
SELECT * FROM ranked


),

-- TODO: make this table actually historical
historical AS (
    SELECT
        *,
        TRUE AS _is_current,
        CAST(universal_first_val AS TIMESTAMP) AS _valid_from,
        
TIMESTAMP_SUB(CAST("2099-01-01" AS TIMESTAMP), INTERVAL 1 MICROSECOND)
 AS _valid_to
    FROM latest_contracts
),

int_transit_database__contracts_dim AS (
    SELECT
        
    
to_hex(md5(cast(coalesce(cast(id as STRING), '') || '-' || coalesce(cast(_valid_from as STRING), '') as STRING))) AS key,
        id AS source_record_id,
        name,
        contract_holder_organization_key,
        contract_vendor_organization_key,
        value,
        start_date,
        end_date,
        covered_services,
        renewal_option,
        notes,
        attachments,
        contract_name_notes,
        covered_components,
        _is_current,
        _valid_from,
        _valid_to
    FROM historical
)

SELECT * FROM int_transit_database__contracts_dim
"

In [ ]:
int_transit_database__contracts_dim = dbGetQuery(con, int_transit_database__contracts_dim_query)

In [ ]:
dim(int_transit_database__contracts_dim)